Importing required python libraries



In [29]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import *
import numpy
from math import log,sqrt
import pandas as pd
import timeit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Defining methods for preprocessing

In [30]:
def tokenize_sentence(sentence):    # Tokenize a string and remove punctuations
#arguments : string
  sentence=sentence.lower()
  tokenizer=TreebankWordTokenizer()
  tokens_list_with_punct = tokenizer.tokenize(sentence.lower())
  tokens_list_without_punct=[]
  for x in tokens_list_with_punct:
   if x.isalpha():
    tokens_list_without_punct.append(x)
  return tokens_list_without_punct

def remove_stopwords(words):        # Remove english stop words from a string
#arguments : List of strings
 stopwords_list=stopwords.words('english')
 filtered_words=[]
 for x in words:
  if x not in stopwords_list:
   filtered_words.append(x)
 return filtered_words

def stem_words(words_list):         # Stem words in a string
#arguments : List of strings
  ps=PorterStemmer()
  stemmed_words=[]
  for x in range(len(words_list)):
   stemmed_words.append(ps.stem(words_list[x]))
  return stemmed_words

def preprocess_sentence(sentence):  # Method to call above defined preprocessing tasks on a string
#arguments : string
  return stem_words(remove_stopwords(tokenize_sentence(sentence)))

Vector space model defined by class " IR_model "

In [31]:
class IR_model():
 def __init__(self):
# constructor method
  self.__word_dict={}      #dictionary of all words in corpus.Each unique word is mapped to an index
  self.__word_dict_size=0  #size of word_dict
  self.__doc_list_size=0   #number of docs in corpus
  self.__score_matrix=None #2D matrix to store tf-idf score
 
 def __addDocument(self,document): #adds document to the matrix and store tf in cells
 # arguments : corpus as a list of tuples ; No return type. 
  for word in document[1]:
   self.__score_matrix[self.__doc_list_size][self.__word_dict[word]]+=1
  self.__doc_list_size+=1

 def build_Vector_Space(self,documents): # computes values for cells in vector space
 #arguments : corpus as a list of tuples ; No return type. 
  for document in documents:             # assign index to unique words
   for word in document[1]:
    if word not in self.__word_dict:
     self.__word_dict[word]=self.__word_dict_size
     self.__word_dict_size+=1

  self.__score_matrix=numpy.zeros((len(documents),self.__word_dict_size))
		
  for document in documents:              # assign tf value to cells
   self.__addDocument(document)

  idf=numpy.zeros((self.__word_dict_size))
  df=numpy.zeros((self.__word_dict_size))

  for word in self.__word_dict:            # calculate df of all words in corpus
   x=self.__word_dict[word]
   for i in range(len(self.__score_matrix)):
    if self.__score_matrix[i][x]!=0:
     df[x]+=1

  for i in range(self.__word_dict_size):   # calculate idf of all words in corpus
   idf[i]=log(len(self.__score_matrix)/df[i])

  for i in range(len(self.__score_matrix)):# fill cells with tf-idf score
   for j in range(self.__word_dict_size):
    if self.__score_matrix[i][j]!=0:
     self.__score_matrix[i][j]=(1+log(self.__score_matrix[i][j]))*(idf[j])
		
 def Search(self,query,documents): # Finds the tf_idf score of query for all docs in corpus and returns it
#arguments : user query as list of terms, corpus as list of tuples
#Return type : list of tuples
  query_df=numpy.zeros((self.__word_dict_size))
  query_idf=numpy.zeros((self.__word_dict_size))
  for word in query:
   if word in self.__word_dict:               # for each word in query, find df and idf
    if query_df[self.__word_dict[word]]==0:
     for document in documents:
      if word in document[1]:
       query_df[self.__word_dict[word]]+=1
       query_idf[self.__word_dict[word]]=log(len(self.__score_matrix)/query_df[self.__word_dict[word]])

  query_score=[]  # To store score between query and each document
  for document in documents:  # Finding tf for each term in query for each doc and find score
   query_tf=numpy.zeros(self.__word_dict_size)
   score=0
   for word in query:
    if word in self.__word_dict:
     if query_tf[self.__word_dict[word]]==0:
      for term in document[1]:
       if term==word:
        query_tf[self.__word_dict[word]]+=1
      if query_tf[self.__word_dict[word]] > 0 :
       query_tf[self.__word_dict[word]]=1+log(query_tf[self.__word_dict[word]])
       score+= query_tf[self.__word_dict[word]] * query_idf[self.__word_dict[word]] 
                                              # score(query,doc)=sum of score for all terms in query and doc
   query_score.append([(score,document[0])]) # append doc id and score for returning
   
  return query_score 


Mounting dataset from google drive

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setting filepath to the dataset

In [33]:
filepath = "/content/drive/My Drive/Colab Notebooks/Song.csv"

Reading the dataset into df

In [34]:
df = pd.read_csv(filepath, error_bad_lines=False)
print(len(df))
df

4049


,Unnamed: 0,Id,Artist Name,Song Name,Lyrics,Clean Lyrics
0,0,1000,Josh Groban,Brave,"Wake up, wake up, the sun cannot wait for long.\nReach out, reach out before it fades away.\nYou will find the warmth when you surrender.\nSmile into the fear and let it play.\n\nYou wanna run away, run away and you say that it can't be so.\nYou wanna look away, look away but you stay cause' it's all so close.\nWhen you stand up and hold out your hand.\nIn the face of what I don't understand.\nMy reason to be brave.\n\nHold on, hold on, so strong, time just carries on.\nAnd all that you thought was wrong is pure again.\nYou can't hide forever from the thunder.\nLook into the storm and feel the rain.\n\nYou wanna run away, run away and you say that it can't be so.\nYou wanna look away, look away but you stay cause' it's all so close.\nWhen you stand up and hold out your hand.\nIn the face of what I don't understand.\nMy reason to be brave.\n\nWhoa (x 4)\n\nGo on, go on…\n\nYou wanna run away, run away and you say that it can't be so.\nYou wanna look away, look away but you stay caus...",wake wake sun wait long reach reach fade away find warmth surrender smile fear let play wan run away run away say cant wan look away look away stay cause close stand hold hand face dont understand reason brave hold hold strong time carry think wrong pure cant hide forever thunder look storm feel rain wan run away run away say cant wan look away look away stay cause close stand hold hand face dont understand reason brave whoa wan run away run away say cant wan look away look away stay cause close stand hold hand face dont understand reason brave explain request
1,1,1001,Simple Plan,Where Is The Love?,"[Originally by Black Eyed Peas]\n\nWhat's wrong with the world mama,\nPeople livin' like ain't got no mamas.\nI think the whole world's addicted to the drama,\nOnly attracted to the things that breathe the trauma.\nOverseas yeah we tryin' to stop terrorism,\nBut we still got terrorists here livin'\nIn the USA, the big CIA, the bloods and the crips and the KKK.\n\nPeople killin' people dyin'\nChildren hurt, do you hear them cryin'?\nIf you practice what you preach?\nWill you turn the other cheek?\nFather, Father, Father help us.\nSend some guidance from above.\nPeople got me got me questioning,\nWhere is the love? (the love, the love)\nWhere is the love, the love, the love?\n\nWhatever happened to the values of humanity?\nWhatever happened to the fairness in equality?\nInstead of spreadin' love we're spreadin' animosity.\nThe lack of understanding leading us away from unity.\nThat's the reason why sometimes I'm feelin' under,\nThat's the reason why sometimes I'm feelin' down,\nIt's ...",originally black eye peas whats wrong world mama people livin like aint get mamas think whole worlds addict drama attract things breathe trauma overseas yeah tryin stop terrorism still get terrorists livin usa big cia blood crips kkk people killin people dyin children hurt hear cryin practice preach turn cheek father father father help send guidance people get get question love love love love love love whatever happen value humanity whatever happen fairness equality instead spreadin love spreadin animosity lack understand lead away unity thats reason sometimes feelin thats reason sometimes feelin wonder sometimes feelin get keep faith alive till love find people killin people dyin children hurt hear cryin practice preach turn cheek father father father help send guidance people get get question love love love love love love love love love love love love love love love love explain request
2,2,1002,Ozzy Osbourne,Paranoid,"Finished with my woman 'cause she couldn't help me with my mind\nPeople think I'm insane because I am frowning all the time\nAll day long I think of things but nothing seems to satisfy\nThink I'll lose my mind if I don't find something to pacify\nCan you help me occupy my brain?\nOh yeah\nI need someone to show me the things in life that I can't find\nI can't see the 

Appending lyrics from the dataset to the list "songs"


In [35]:
songs=[]     #"songs" is a list of lyrics from df
for i in df['Clean Lyrics']:
 songs.append(i)

Generating the corpus in required form.

In [36]:
docs=[]#"docs" is a list where each element in the list is a tuple whose first element is index of song in corpus and second element is preprocessed item
start = timeit.default_timer()

docs = [(i, preprocess_sentence(songs[i])) for i in range(len(songs))]

stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  12.86841538300007


Building the IR model.


In [37]:
start = timeit.default_timer()
Model=IR_model()      #"Model" is the Vector space model.
Model.build_Vector_Space(docs)
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  145.14310020599999


Assigning the query to q.

In [56]:
start = timeit.default_timer()
q="startt"
q=preprocess_sentence(q)
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.000857245000020157


Calculating tf_idf score for query and each song.

In [57]:
start = timeit.default_timer()
f=Model.Search(q,docs)
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.04730948500002796


Getting the index of top 10 similar songs in corpus.

In [58]:
isallzeros = 1
f.sort(reverse=True)
top10=[]
for i in range(10):
  for j in f[i]:
      if j[0] > 0:
        isallzeros = 0
if isallzeros == 1 :
 result="Sorry, no match found"
else:     
 for i in range(10):
  for j in f[i]:
   top10.append(j[1])

Appending other details of top 10 songs in ans.

In [59]:
if isallzeros == 0 :
 ans=[]
 for i in top10:
  ans.append([df.iloc[i]['Artist Name'],df.iloc[i]['Song Name'],df.iloc[i]['Clean Lyrics']])

Printing top 10 relevant results.

In [60]:
if isallzeros == 0 :
 pd.set_option('display.max_colwidth',1000)
 print("Top 10 search results\n")
 result=pd.DataFrame(ans,columns=['Artist Name','Song Name','Lyrics'])
 result

In [61]:
result

'Sorry, no match found'